In [81]:
import pandas as pd

tournaments = pd.read_csv('./Data Files/tournament.csv')
players = pd.read_csv('./Data Files/player.csv')
matches = pd.read_csv('./Data Files/match.csv')
profiles = pd.read_csv('./Data Files/playerProfile.csv')
country = pd.read_csv('./Data Files/country.csv')
arbiter = pd.read_csv('./Data Files/arbiter.csv')

In [82]:
import numpy as np

In [83]:
from borb.pdf import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF
from borb.pdf.canvas.layout.page_layout.multi_column_layout import SingleColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout
from decimal import Decimal
from borb.pdf.canvas.layout.image.image import Image

In [84]:
from borb.pdf import FixedColumnWidthTable, FlexibleColumnWidthTable

In [85]:
from borb.pdf.canvas.layout.text.paragraph import Paragraph

In [86]:
from pathlib import Path

In [87]:
from borb.pdf import HexColor

In [88]:
from borb.pdf.canvas.font.simple_font.true_type_font import TrueTypeFont

In [89]:
import requests

In [90]:
from borb.pdf.canvas.font.font import Font
from borb.pdf import Alignment
from borb.pdf import TableCell

In [91]:
font_path: Path = Path('./Assets/Fonts/Everson Mono.ttf')
custom_font: Font = TrueTypeFont.true_type_font_from_file(font_path)

In [92]:
imgAttrib = 'image: Flaticon.com'

In [93]:
pdf = Document()
page = Page()

pdf.add_page(page)

layout = SingleColumnLayout(page)

layout.add(    
    Image(
        Path('./Assets/strategy.png'),        
    width=Decimal(128),        
    height=Decimal(128), horizontal_alignment=Alignment.CENTERED)
    )

layout.add(Paragraph('Gambit Chess Tournament Report', font_size=22, font=custom_font, horizontal_alignment=Alignment.CENTERED))

In [94]:
tournamentID = 'T-883-QMFJ'

tournament = tournaments[tournaments['Tournament_ID'] == tournamentID]
matchesTourn = matches[matches['Tournament_ID'] == tournamentID]

tournName = tournament['Tournament_Name'].values[0]
startDate = tournament['Start_Date'].values[0]
endDate = tournament['End_Date'].values[0]
winner = tournament['Winner'].values[0]
loc = tournament['Location'].values[0]
countryName = country[country['Country_ID'] == tournament['Country'].values[0]]['Country_Name'].values[0]

In [95]:
layout.add(Paragraph(''))
layout.add(Paragraph(''))
layout.add(Paragraph(''))
layout.add(Paragraph(''))

layout.add(Paragraph('Tournament Name: ' + tournName, font_size=14, font=custom_font))
layout.add(Paragraph('Start Date: ' + startDate, font_size=14, font=custom_font))
layout.add(Paragraph('End Date: ' + endDate, font_size=14, font=custom_font))
layout.add(Paragraph('Location: ' + loc, font_size=14, font=custom_font))
layout.add(Paragraph('Country: ' + countryName, font_size=14, font=custom_font))

In [96]:
page2 = Page()

pdf.add_page(page2)

layout2 = SingleColumnLayout(page2)

layout2.add(Paragraph('Host', font_size=12, font=custom_font))
layout2.add(Paragraph('Country: ' + countryName, font_size=12, font=custom_font))
layout2.add(Paragraph('Organization: ' + country[country['Country_ID'] == tournament['Country'].values[0]]['Chess_Org'].values[0], font_size=12, font=custom_font))
layout2.add(Paragraph('International License: ' + country[country['Country_ID'] == tournament['Country'].values[0]]['Org_License'].values[0], font_size=12, font=custom_font))
layout2.add(Paragraph('President: ' + country[country['Country_ID'] == tournament['Country'].values[0]]['Org_President'].values[0], font_size=12, font=custom_font))
layout2.add(Paragraph(''))

layout2.add(Paragraph('Winner', font_size=12, font=custom_font))
layout2.add(Paragraph('License: ' + winner, font_size=12, font=custom_font))
layout2.add(Paragraph('Name: ' + players[players['Player_ID'] == winner]['Player_Name'].values[0], font_size=12, font=custom_font))
layout2.add(Paragraph('Country: ' + country[country['Country_ID'] == players[players['Player_ID'] == winner]['Country'].values[0]]['Country_Name'].values[0], font_size=12, font=custom_font))
layout2.add(Paragraph('Date of Birth: ' + str(players[players['Player_ID'] == winner]['DOB'].values[0]), font_size=12, font=custom_font))
layout2.add(Paragraph('Rating: ' + str(players[players['Player_ID'] == winner]['Player_Rating'].values[0]), font_size=12, font=custom_font))
layout2.add(Paragraph('FIDE Percentile: ' + str(players[players['Player_ID'] == winner]['FIDE_Percentile'].values[0]), font_size=12, font=custom_font))
layout2.add(Paragraph(''))
layout2.add(Paragraph('Achievements', font_size=12, font=custom_font))

prof = profiles[profiles['Player_ID'] == winner]
prof = prof[prof['Achievements'].astype(str) != 'nan']

table = FixedColumnWidthTable(number_of_rows=len(prof) + 1, number_of_columns=3)

odd_color = HexColor("BBBBBB")
even_color = HexColor("FFFFFF")

table.add(TableCell(Paragraph('Achievement', font_size=12, font=custom_font), background_color=odd_color, padding_top=Decimal(5), padding_left=Decimal(5)))
table.add(TableCell(Paragraph('Year', font_size=12, font=custom_font), background_color=odd_color, padding_top=Decimal(5), padding_left=Decimal(5)))
table.add(TableCell(Paragraph('Coach', font_size=12, font=custom_font), background_color=odd_color, padding_top=Decimal(5), padding_left=Decimal(5)))
    
for i in range(len(prof)):
    c = even_color if i % 2 == 0 else odd_color
    row = prof.iloc[i]
    table.add(TableCell(Paragraph(row['Achievements'][:-3], font_size=10, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))
    table.add(TableCell(Paragraph(str(row['Year']), font_size=10, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))
    table.add(TableCell(Paragraph(row['CoachName'], font_size=10, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))

layout2.add(table)

layout2.add(Paragraph(''))
layout2.add(Paragraph(''))

In [97]:
odd_color = HexColor("BBBBBB")
even_color = HexColor("FFFFFF")
w = HexColor("FBEB70")

columns = ['Match ID', 'Player White', 'Player Black', 'Arbiter', 'Round']

def createTable(data):
    table = FixedColumnWidthTable(number_of_rows=len(data) + 1, number_of_columns=len(columns))

    for column in columns:
        table.add(TableCell(Paragraph(column, font_size=12, font=custom_font), background_color=odd_color, padding_top=Decimal(5), padding_left=Decimal(5)))

    for i in range(len(data)):
        c = even_color if i % 2 == 0 else odd_color
        row = data.iloc[i]

        table.add(TableCell(Paragraph(str(row['Match_ID']), font_size=8, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))

        if row['Player_Black'] == row['Winner']:
            table.add(TableCell(Paragraph(row['Player_White'], font_size=8, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))
            table.add(TableCell(Paragraph(row['Player_Black'], font_size=8, font=custom_font), background_color=w, padding_top=Decimal(5), padding_left=Decimal(5)))
        elif row['Player_White'] == row['Winner']:
            table.add(TableCell(Paragraph(row['Player_White'], font_size=8, font=custom_font), background_color=w, padding_top=Decimal(5), padding_left=Decimal(5)))
            table.add(TableCell(Paragraph(row['Player_Black'], font_size=8, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))

        table.add(TableCell(Paragraph(row['Arbiter'], font_size=8, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))
        table.add(TableCell(Paragraph(str(row['Round']), font_size=8, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))

    return table

def below32():
    page3 = Page()

    pdf.add_page(page3)

    layout3 = SingleColumnLayout(page3)

    layout3.add(Paragraph('Matches', font_size=14, font=custom_font))

    layout3.add(createTable(matchesTourn))

    layout3.add(Paragraph(''))
    layout3.add(Paragraph('Note: Winner of match is highlighted', font_size=6, font=custom_font))
    layout3.add(Paragraph('All IDs can be referenced in the appendix', font_size=6, font=custom_font))

def above32():
    page3 = Page()

    pdf.add_page(page3)

    layout3 = SingleColumnLayout(page3)

    layout3.add(Paragraph('Matches', font_size=14, font=custom_font))

    layout3.add(createTable(matchesTourn.iloc[:32]))

    pageT = Page()

    pdf.add_page(pageT)

    layoutT = SingleColumnLayout(pageT)

    layoutT.add(Paragraph('Matches - Contd.', font_size=14, font=custom_font))
    
    layoutT.add(createTable(matchesTourn.iloc[32:].reset_index(drop=True)))

    layoutT.add(Paragraph('Note: Winner of match is highlighted', font_size=6, font=custom_font))
    layoutT.add(Paragraph('All IDs can be referenced in the appendix', font_size=6, font=custom_font))

In [98]:
if len(matchesTourn) > 32:
    above32()
else:
    below32()

In [99]:
page4 = Page()

pdf.add_page(page4)

layout4 = SingleColumnLayout(page4)

layout4.add(Paragraph('Appendix', font_size=14, font=custom_font))

layout4.add(Paragraph('Players', font_size=12, font=custom_font))

plys = set(matchesTourn['Player_Black'].to_list() + matchesTourn['Player_White'].to_list())

for ply in plys:
    layout4.add(Paragraph(ply, font_size=8, font=custom_font, font_color=HexColor("FF0000")))
    row = players[players['Player_ID'] == ply]
    layout4.add(Paragraph('Name: ' + row['Player_Name'].values[0], font_size=8, font=custom_font))
    layout4.add(Paragraph('Country: ' + country[country['Country_ID'] == row['Country'].values[0]]['Country_Name'].values[0], font_size=8, font=custom_font))
    layout4.add(Paragraph('FIDE_Percentile - Rating: ' + str(row['FIDE_Percentile'].values[0]) + ' - ' + str(row['Player_Rating'].values[0]), font_size=8, font=custom_font))
    layout4.add(Paragraph('', font_size=8, font=custom_font))

layout4.add(Paragraph(''))

layout4.add(Paragraph('Arbiters', font_size=12, font=custom_font))

for arb in matchesTourn['Arbiter'].unique():
    row = arbiter[arbiter['Arbiter_ID'] == arb]
    
    layout4.add(Paragraph(arb + ' : ' + row['Arbiter_Name'].values[0], font_size=8, font=custom_font))


In [100]:
with open('./Assets/Outs/test.pdf', 'wb') as pdfOut:
    PDF.dumps(pdfOut, pdf)